<a href="https://colab.research.google.com/github/ZikrullaRaxmatov/ML_DL_project/blob/main/segment_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import cv2
import os

In [37]:
model = YOLO('weights/yolov8m-seg.pt')
names = model.model.names

In [46]:
input_video_path = 'videos/night_one_way_traffic.mp4'
output_video_path = f'out/segment_{os.path.basename(input_video_path)}'

In [47]:
cap = cv2.VideoCapture(input_video_path)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (w, h))

selected_names = [
    list(names.values()).index('car'),
    list(names.values()).index('truck'),
]

print(selected_names)

[2, 7]


In [ ]:
while True:
  ret, frame = cap.read()

  if not ret:
    print('Video yakunlandi!!!')
    break

  results = model.predict(frame)
  clss = results[0].boxes.cls.cpu().tolist()
  masks = results[0].masks.xy

  annotator = Annotator(frame, line_width=3)

  for mask, cls in zip(masks, clss):
    if int(cls) in selected_names:
      try:
        annotator.seg_bbox(mask=mask, mask_color=(0, 255, 0), label=names[int(cls)])
      except: continue

  out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()